# Appendix B Python Code

Section 1: Here we have the code needed to create a Convolutional Neural Network for classifying a flower image dataset. 

In [ ]:
#Here we have all the packages needed to create our Neural Net
import os
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import time
from sklearn.cross_validation import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
import itertools

Section 2: Here we upload the images and resize them to by 100 by 100 by 3

In [ ]:
#this function just takes the images and opens them, resizes them, 
#and puts them in an array
def LoadDir(dirname):
    imgs = []
    for imgname in os.listdir(dirname):
        img = Image.open(os.path.join(dirname, imgname))
        img = img.resize([100, 100])
        img = np.squeeze(np.array(img)[:, :, :])
        imgs.append(img)
    return np.array(imgs)
#here I load the images 
daisy_imgs = LoadDir('C:\\Users\\ebish\\Documents\\data analysis 582\\flowers\\daisy')
print(daisy_imgs.shape)
dandelion_imgs = LoadDir('C:\\Users\\ebish\\Documents\\data analysis 582\\flowers\\dandelion')
print(dandelion_imgs.shape)
rose_imgs = LoadDir('C:\\Users\\ebish\\Documents\\data analysis 582\\flowers\\rose')
print(rose_imgs.shape)
sunflower_imgs = LoadDir('C:\\Users\\ebish\\Documents\\data analysis 582\\flowers\\sunflower')
print(sunflower_imgs.shape)
tulip_imgs = LoadDir('C:\\Users\\ebish\\Documents\\data analysis 582\\flowers\\tulip')
print(tulip_imgs.shape)

Section 3: Here we create a plot of some of the images in the dataset just to get an idea of what we are working with.

In [ ]:
#plotting the different images
plt.subplot(1,5,1)
plt.imshow(daisy_imgs[1])
plt.axis('off')
plt.subplot(1,5,2)
plt.imshow(dandelion_imgs[1])
plt.axis('off')
plt.subplot(1,5,3)
plt.imshow(rose_imgs[1])
plt.axis('off')
plt.subplot(1,5,4)
plt.imshow(sunflower_imgs[1])
plt.axis('off')
plt.subplot(1,5,5)
plt.imshow(tulip_imgs[8])
plt.axis('off')

Section 4: We normalize the images and then create labels for the flowers and combine all of it together. 

In [ ]:
#normalize the data
daisy_imgs=daisy_imgs/255
dandelion_imgs=dandelion_imgs/255
rose_imgs=rose_imgs/255
sunflower_imgs=sunflower_imgs/255
tulip_imgs=tulip_imgs/255
#creating the labels for the data by finding their lenght and then 
#creating an array that holds thier class as an integer
total=len(daisy_imgs)+len(dandelion_imgs)+len(rose_imgs)+len(sunflower_imgs)+len(tulip_imgs)
y=np.zeros(shape=(total,1))
a=len(daisy_imgs)
b=len(dandelion_imgs)
c=len(rose_imgs)
d=len(sunflower_imgs)
e=len(tulip_imgs)
y[a:a+b-1]=1; #dandelions
y[a+b:a+b+c-1]=2; #rose
y[a+b+c:a+b+c+d-1]=3; #sunflower
y[a+b+c+d:a+b+c+d+e-1]=4; #tulip
#combine all the flower data
allflowers=np.vstack([daisy_imgs, dandelion_imgs, rose_imgs, sunflower_imgs,tulip_imgs])


Section 5: Created the training, validation, and test dataset

In [ ]:
X_train, X_testval, y_train,y_testval=train_test_split(allflowers,y,test_size=.20)
X_validate, X_test,  y_validate, y_test=train_test_split(X_testval, y_testval, test_size=.5)

In [ ]:
#our nerual net will take the y dataset and turn in essetially into
#one hot encoding
y_train=to_categorical(y_train, num_classes=5)
y_test=to_categorical(y_test, num_classes=5)
y_validate=to_categorical(y_validate, num_classes=5)

Section 6: Here we created an initial convolutional nerual net. We have 2 convoltuional layers with maxpooling layers followed after each one. Then we go straight to the output layer.

In [ ]:
#initial model has 2 convolutaional layers where our filter is (3,3)
#with a RELU activation function. Two maxpooling layers that follow
#each convolutional layer with a filter of the size (2,2)
#No dense layer other than to compute the output with a softmax 
#activation function. Loss=categorical crossentropy with a Adam 
#optimizer.
model1=Sequential()
model1.add( Conv2D(64,(3,3), input_shape=X_train.shape[1:]) )
model1.add(Activation("relu"))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add( Conv2D(64,(3,3) ))
model1.add(Activation("relu"))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Flatten())

model1.add(Dense(5))
model1.add(Activation('softmax'))

model1.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

model1.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=.1, callbacks=[tensorboard])

Section 7: Here we create the loop that tests out every combination of dense layer, nodes, and convolutional layers. 

In [1]:
#different testing values
dense_layers=[0, 1, 2]
layer_sizes=[32, 64, 128]
conv_layers=[1,2,3]

#for loop to try out the different models where it will try every
#combination of dense_layers, layer_sizes, and conv_layers.
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            #this connects with tensorboard and allows us to plot the 
            #accuracy of each model that we iterate through
            NAME="{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))
            print(NAME)
            model1=Sequential()
            model1.add( Conv2D(layer_size,(3,3), input_shape=X_train.shape[1:]) )
            model1.add(Activation("relu"))
            model1.add(MaxPooling2D(pool_size=(2,2)))

            for l in range(conv_layer-1):
                model1.add( Conv2D(layer_size,(3,3) ))
                model1.add(Activation("relu"))
                model1.add(MaxPooling2D(pool_size=(2,2)))
                
            model1.add(Flatten())    
            for l in range(dense_layer):
                model1.add(Dense(layer_size))
                model1.add(Activation("relu"))

            model1.add(Dense(5))
            model1.add(Activation('softmax'))
            #compile the model
            model1.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])
            #train the model with the given data 
            model1.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=.1, callbacks=[tensorboard])

NameError: name 'time' is not defined

Section 8: Here we prep to do data augmentation 

In [ ]:
#Here we do some data augmentation to help us have more training samples
X_train_ch=X_train
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


d=datagen.fit(X_train_ch)

Section 9: Here we create our model1, model2, and model3 with newly added dropout and data augmentation

In [ ]:
#model 1
NAME="{}-conv-{}-nodes-{}-dense-{}-dropout-dgm".format(3, 32, 0, int(time.time()))
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))
print(NAME)
model1=Sequential()
model1.add( Conv2D(32,(3,3), input_shape=X_train.shape[1:]) )
model1.add(Activation("relu"))
model1.add(MaxPooling2D(pool_size=(2,2)))

           
model1.add( Conv2D(32,(3,3) ))
model1.add(Activation("relu"))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add( Conv2D(32,(3,3) ))
model1.add(Activation("relu"))
model1.add(MaxPooling2D(pool_size=(2,2)))
                
model1.add(Flatten())
model1.add(Dropout(0.2))

model1.add(Dense(5))
model1.add(Activation('softmax'))

model1.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

model1.fit_generator(datagen.flow(X_train_ch,y_train, batch_size=32),
                              epochs = 10, validation_data=(X_validate,y_validate), callbacks=[tensorboard])
                                                     

In [ ]:
#model 2
NAME="{}-conv-{}-nodes-{}-dense-{}-dropout-dgm2".format(3, 64, 1, int(time.time()))
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))
print(NAME)
model2=Sequential()
model2.add( Conv2D(64,(3,3), input_shape=X_train.shape[1:]) )
model2.add(Activation("relu"))
model2.add(MaxPooling2D(pool_size=(2,2)))

           
model2.add( Conv2D(64,(3,3) ))
model2.add(Activation("relu"))
model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add( Conv2D(64,(3,3) ))
model2.add(Activation("relu"))
model2.add(MaxPooling2D(pool_size=(2,2)))
                
model2.add(Flatten())    
model2.add(Dense(64))
model2.add(Activation("relu"))
model2.add(Dropout(0.2))

model2.add(Dense(5))
model2.add(Activation('softmax'))

model2.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

model2.fit_generator(datagen.flow(X_train_ch,y_train, batch_size=32),
                              epochs = 10, validation_data=(X_validate,y_validate), callbacks=[tensorboard])
                              

In [ ]:
#model 3
NAME="{}-conv-{}-nodes-{}-dense-{}-dropout-dgm2".format(3, 64, 2, int(time.time()))
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))
print(NAME)
model3=Sequential()
model3.add( Conv2D(64,(3,3), input_shape=X_train.shape[1:]) )
model3.add(Activation("relu"))
model3.add(MaxPooling2D(pool_size=(2,2)))

           
model3.add( Conv2D(64,(3,3) ))
model3.add(Activation("relu"))
model3.add(MaxPooling2D(pool_size=(2,2)))

model3.add( Conv2D(64,(3,3) ))
model3.add(Activation("relu"))
model3.add(MaxPooling2D(pool_size=(2,2)))
                
model3.add(Flatten())    
model3.add(Dense(64))
model3.add(Activation("relu"))

model3.add(Dense(64))
model3.add(Activation("relu"))
model3.add(Dropout(0.2))

model3.add(Dense(5))
model3.add(Activation('softmax'))

model3.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

model3.fit_generator(datagen.flow(X_train_ch,y_train, batch_size=32),
                              epochs = 10, validation_data=(X_validate,y_validate), callbacks=[tensorboard])
                            

Section 10: Here we test different acitvaiton functions on our best model from Section 9. 

In [ ]:
#testing different activaiton functions 
act=["sigmoid","tanh", "relu"]


for act_layer in act:
    NAME="{}-conv-{}-nodes-{}-dense-{}-dropout-{}-activation-dgm2".format(3, 64, 2,act_layer, int(time.time()))
    tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))
    print(NAME)
    model4=Sequential()
    model4.add( Conv2D(64,(3,3), input_shape=X_train.shape[1:]) )
    model4.add(Activation(act_layer))
    model4.add(MaxPooling2D(pool_size=(2,2)))


    model4.add( Conv2D(64,(3,3) ))
    model4.add(Activation(act_layer))
    model4.add(MaxPooling2D(pool_size=(2,2)))

    model4.add( Conv2D(64,(3,3) ))
    model4.add(Activation(act_layer))
    model4.add(MaxPooling2D(pool_size=(2,2)))

    model4.add(Flatten())    
    model4.add(Dense(64))
    model4.add(Activation(act_layer))
    model4.add(Dropout(0.2))
    
    model4.add(Dense(64))
    model4.add(Activation(act_layer))
   
    
    model4.add(Dense(5))
    model4.add(Activation('softmax'))

    model4.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

    model4.fit_generator(datagen.flow(X_train_ch,y_train, batch_size=32),
                                  epochs = 5, validation_data=(X_validate,y_validate), callbacks=[tensorboard])


Section 11: now test between our two final models with 3 convolutional layers vs 4 convolutional layers. We test to see if more epochs will increase the overall accuracy on our validation dataset. 

In [ ]:
NAME="{}-conv-{}-nodes-{}-dense-{}-dropout-{}-activation-dgm2".format(3, 64, 2,"relu", int(time.time()))
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))
print(NAME)
model6=Sequential()
model6.add( Conv2D(64,(3,3), input_shape=X_train.shape[1:]) )
model6.add(Activation("relu"))
model6.add(MaxPooling2D(pool_size=(2,2)))


model6.add( Conv2D(64,(3,3) ))
model6.add(Activation("relu"))
model6.add(MaxPooling2D(pool_size=(2,2)))

model6.add( Conv2D(64,(3,3) ))
model6.add(Activation("relu"))
model6.add(MaxPooling2D(pool_size=(2,2)))

model6.add(Flatten())    
model6.add(Dense(64))
model6.add(Activation("relu"))
model6.add(Dropout(0.2))
    
model6.add(Dense(64))
model6.add(Activation("relu"))
   
    
model6.add(Dense(5))
model6.add(Activation('softmax'))

model6.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

model6.fit_generator(datagen.flow(X_train_ch,y_train, batch_size=32),
                                  epochs = 25, validation_data=(X_validate,y_validate), callbacks=[tensorboard])

In [ ]:
NAME="{}-conv-{}-nodes-{}-dense-{}-dropout-{}-activation-dgm2".format(4, 64, 2,act_layer, int(time.time()))
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))
print(NAME)
model7=Sequential()
model7.add( Conv2D(64,(3,3), input_shape=X_train.shape[1:]) )
model7.add(Activation(act_layer))
model7.add(MaxPooling2D(pool_size=(2,2)))


model7.add( Conv2D(64,(3,3) ))
model7.add(Activation(act_layer))
model7.add(MaxPooling2D(pool_size=(2,2)))

model7.add( Conv2D(64,(3,3) ))
model7.add(Activation(act_layer))
model7.add(MaxPooling2D(pool_size=(2,2)))

model7.add( Conv2D(64,(3,3) ))
model7.add(Activation("relu"))
model7.add(MaxPooling2D(pool_size=(2,2)))

model7.add(Flatten())    
model7.add(Dense(64))
model7.add(Activation(act_layer))
model7.add(Dropout(0.2))
    
model7.add(Dense(64))
model7.add(Activation(act_layer))
   
    
model7.add(Dense(5))
model7.add(Activation('softmax'))

model7.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

model7.fit_generator(datagen.flow(X_train_ch,y_train, batch_size=32),
                                  epochs = 25, validation_data=(X_validate,y_validate), callbacks=[tensorboard])


Section 12:We have finally picked a final model. Model 7 is the best that we have. Here we create a confusion matrix to try to understand the misclassification 

In [ ]:
y_pred_test_model7=model7.predict_classes(X_test)
y_test_returned=np.argmax(y_test,axis=1)

In [ ]:
#this function creates a confusion matrix. 
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.rcParams.update({'font.size':22})
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.rcParams.update({'font.size':32})
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test_returned, y_pred_test_model7);
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10))
plot_confusion_matrix(cnf_matrix, classes=['daisy','dandelion','rose','sunflower','tulip'],
                      title='Confusion matrix, with normalization')
plt.show()

Section 13: Create the classifcation chart

In [ ]:
print(classification_report(y_test_returned, y_pred_test_model7,target_names=['daisy','dandelion','rose','sunflower','tulip']))

Section 14: This was my way of finding the misclassified images and then plotting them to try to better understand what was going on.  

In [ ]:
print(y_pred_test_model7[10:20])
print(y_test_returned[10:20])

In [ ]:
#plots different images of the flowers from the test data to be able 
#to try to understand what is going on with the data 
#Tulip to rose 4 to 2
plt.rcParams.update({'font.size':8})
first=plt.subplot(1,4,1)
first.title.set_text('Predicted:Tulip True Label:Rose')
plt.imshow(X_test[19])
plt.axis('off')
plt.subplot(1,4,2)
plt.imshow(X_test[20])
plt.axis('off')

plt.subplot(1,4,3)
plt.imshow(X_test[205])
plt.axis('off')
plt.subplot(1,4,4)
plt.imshow(X_test[242])
plt.axis('off')



In [ ]:
#opposite Rose to tulip 2 to 4

second=plt.subplot(1,4,1)
second.title.set_text('Predicted:Rose vs. True Label:Tulip')
plt.imshow(X_test[27])
plt.axis('off')
plt.subplot(1,4,2)
plt.imshow(X_test[114])
plt.axis('off')
plt.subplot(1,4,3)
plt.imshow(X_test[221])
plt.axis('off')
plt.subplot(1,4,4)
plt.imshow(X_test[276])
plt.axis('off')

In [ ]:
#sunflower to rose 3 to 2
c=plt.subplot(2,4,5)
c.title.set_text('P: Sunflower T: Rose')
plt.imshow(X_test[120])
plt.axis('off')
plt.subplot(2,4,6)
plt.imshow(X_test[201])
plt.axis('off')
#sunflower to tulip 3 to 4
d=plt.subplot(2,4,7)
d.title.set_text('P: Sunflower T: Tulip')
plt.imshow(X_test[25])
plt.axis('off')
plt.subplot(2,4,8)
plt.imshow(X_test[15])
plt.axis('off')